In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
x_train.shape,x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
# Normalization
x_train = x_train/255.0
x_test = x_test/255.0

# **Random Forest**

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Flatten the image data
x_train_flatten = x_train.reshape(x_train.shape[0], -1)
x_test_flatten = x_test.reshape(x_test.shape[0], -1)

# Create and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train_flatten, y_train)

# Make predictions
y_pred = rf_classifier.predict(x_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


<ipython-input-5-1ac9a0dda03b>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(x_train_flatten, y_train)


Accuracy: 0.4654


In [ ]:
accuracy_score(y_pred,y_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.56      0.54      0.55      1028
           1       0.54      0.52      0.53      1046
           2       0.33      0.38      0.35       889
           3       0.28      0.33      0.30       856
           4       0.38      0.39      0.39       977
           5       0.40      0.43      0.41       937
           6       0.57      0.47      0.52      1192
           7       0.45      0.51      0.48       895
           8       0.61      0.58      0.59      1064
           9       0.52      0.47      0.50      1116

    accuracy                           0.47     10000
   macro avg       0.47      0.46      0.46     10000
weighted avg       0.47      0.47      0.47     10000



# **Implementing a Decision Tree**

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Flatten the image data
x_train_flatten = x_train.reshape(x_train.shape[0], -1)
x_test_flatten = x_test.reshape(x_test.shape[0], -1)

# Create and train the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(x_train_flatten, y_train)

# Make predictions
y_pred = dt_classifier.predict(x_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.2663


# **Implementating CNN**

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
# Normalize the image data
x_train_normalized = x_train / 255.0
x_test_normalized = x_test / 255.0

# Define the CNN architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set the number of epochs
epochs = 50

# Callbacks for saving model and TensorBoard logging
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
    tf.keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch"),
]

# Train the model and evaluate simultaneously
history = model.fit(
    x=x_train_normalized,
    y=y_train,
    validation_data=(x_test_normalized, y_test),
    epochs=epochs,
    callbacks=callbacks
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test_normalized, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


Epoch 1/50
1563/1563 [==============================] - 111s 70ms/step - loss: 1.5255 - accuracy: 0.4431 - val_loss: 1.2558 - val_accuracy: 0.5437
Epoch 2/50
1563/1563 [==============================] - 88s 56ms/step - loss: 1.1721 - accuracy: 0.5829 - val_loss: 1.0752 - val_accuracy: 0.6171
Epoch 3/50
1563/1563 [==============================] - 86s 55ms/step - loss: 1.0112 - accuracy: 0.6443 - val_loss: 0.9541 - val_accuracy: 0.6666
Epoch 4/50
1563/1563 [==============================] - 92s 59ms/step - loss: 0.9075 - accuracy: 0.6802 - val_loss: 0.9265 - val_accuracy: 0.6739
Epoch 5/50
1563/1563 [==============================] - 85s 54ms/step - loss: 0.8305 - accuracy: 0.7067 - val_loss: 0.9502 - val_accuracy: 0.6676
Epoch 6/50
1563/1563 [==============================] - 93s 60ms/step - loss: 0.7672 - accuracy: 0.7303 - val_loss: 0.8573 - val_accuracy: 0.7014
Epoch 7/50
1563/1563 [==============================] - 89s 57ms/step - loss: 0.7201 - accuracy: 0.7479 - val_loss: 0.8607 

In [ ]:
import numpy as np
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test_normalized, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Predict class probabilities for the test data
probabilities = model.predict(x_test_normalized)

# Define a threshold for low probability
probability_threshold = 0.2  # Adjust this value as needed

# Generate labels based on probabilities and threshold
labels = []
for prob in probabilities:
    if np.max(prob) >= probability_threshold:
        labels.append(np.argmax(prob))
    else:
        labels.append("Image not in data label")
# Print some example results
for i in range(10):  # Print results for the first 10 images
    true_label = y_test[i]
    predicted_label = labels[i]
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")

In [5]:
model.save("Image_Classification_Model.h5")

In [10]:
# lets explore our metrics a bit
import pandas as pd
Metrics = pd.DataFrame(model.history.history)

In [12]:
Metrics

""


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs

In [13]:
!pip install gradio

In [15]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image

# Load your CNN model
model_path = "Image_Classification_Model.h5"  # Update with your model's path
model = load_model(model_path)

# Define class labels
class_labels = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

# Function to classify images
def classify_image(image):
    # Preprocess the image
    image = image.resize((32, 32))  # Resize to match your model's input size
    image = np.array(image)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)

    # Get the predicted class probabilities
    predictions = model.predict(image)
    predicted_class_probs = predictions[0]

    # Prepare label and probability information
    label_probs = []
    high_prob_found = False
    for idx, prob in enumerate(predicted_class_probs):
        if prob >= 0.3:
            label_name = class_labels[idx]
            if prob > 0.6:  # Highlight high-probability labels
                label_name = f"<span style='color:black; background-color:#FFD700'>{label_name}</span>"
                high_prob_found = True
            label_probs.append(f"{label_name}: {prob:.4f}")

    # If no high probability label found, show "Not in data label"
    if not high_prob_found:
        label_probs = ["Not in data label"]

    # Return the class labels and their probabilities
    return "<br>".join(label_probs)

# Define the Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.inputs.Image(type="pil"),
    outputs=gr.outputs.HTML(),  # Use HTML output for custom font colors
    live=True,
    title="Image Classification with CNN",
    description=f"Available data labels: {', '.join(class_labels.values())}"
)

# Launch the interface
iface.launch()


<ipython-input-15-6c765dc1e031>:46: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(type="pil"),
<ipython-input-15-6c765dc1e031>:46: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(type="pil"),
<ipython-input-15-6c765dc1e031>:47: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(),


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# Launch the interface
iface.launch(share=True)